# How Many Washingtonians are Low- or Middle- Income?

In [126]:
# import necessary packages
import pandas as pd
import numpy as np

## State-level data

Data Source: 2018 American Community Survey via IPUMS NHGIS, University of Minnesota, www.nhgis.org

In [ ]:
states = pd.read_csv('../data/2018_state.csv')

In [ ]:
wa = states.loc[states['STATE'] == 'Washington']

In [4]:
wa

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTYA,COUSUBA,PLACEA,TRACTA,...,AJY9M017,AJZAM001,AJ0EM001,AJ1CM001,AJ1CM002,AJ1CM003,AJ1CM004,AJ1CM005,AJ1CM006,AJ1CM007
48,G530,2014-2018,NaN,NaN,Washington,53,NaN,NaN,NaN,NaN,...,2752,308,168,1574,6117,5945,6919,3629,1425,6190


### ratio of income to poverty level - population for whom poverty status is determined

In [5]:
income_to_pov = wa.loc[:, 'AJY4E001': 'AJY4E008']
income_to_pov

,AJY4E001,AJY4E002,AJY4E003,AJY4E004,AJY4E005,AJY4E006,AJY4E007,AJY4E008
48,7161708,377106,444515,255057,273495,395859,177498,5238178


In [52]:
income_to_pov.columns = ['Total', 
        'Under .50',
        '.50 to .99',
        '1.00 to 1.24',
        '1.25 to 1.49',
        '1.50 to 1.84',
        '1.85 to 1.99',
        '2.00 and over']
income_to_pov.columns = income_to_pov.columns.str.replace(' ', '_').str.lower()

In [53]:
income_to_pov = income_to_pov.astype(int)

In [54]:
income_to_pov

,total,under_.50,.50_to_.99,1.00_to_1.24,1.25_to_1.49,1.50_to_1.84,1.85_to_1.99,2.00_and_over
48,7161708,377106,444515,255057,273495,395859,177498,5238178


In [16]:
less_than_2x_poverty = sum(income_to_pov.loc[48, 'under_half':'1p85_1p99'])

In [17]:
less_than_2x_poverty

1923530

In [18]:
less_than_2x_poverty / income_to_pov['total']

48    0.268585
Name: total, dtype: float64

### household income

In [24]:
house_income = wa.loc[:,'AJY9E001':'AJY9E017'].astype(int)

In [50]:
house_income.columns = ['Total',
        'Less than $10,000',
        '$10,000 to $14,999',
        '$15,000 to $19,999',
        '$20,000 to $24,999',
        '$25,000 to $29,999',
        '$30,000 to $34,999',
        '$35,000 to $39,999',
        '$40,000 to $44,999',
        '$45,000 to $49,999',
        '$50,000 to $59,999',
        '$60,000 to $74,999',
        '$75,000 to $99,999',
        '$100,000 to $124,999',
        '$125,000 to $149,999',
        '$150,000 to $199,999',
        '$200,000 or more']
house_income.columns = house_income.columns.str.replace(' ', '_').str.replace('$', '').str.lower()

In [51]:
house_income

,total,"less_than_10,000","10,000_to_14,999","15,000_to_19,999","20,000_to_24,999","25,000_to_29,999","30,000_to_34,999","35,000_to_39,999","40,000_to_44,999","45,000_to_49,999","50,000_to_59,999","60,000_to_74,999","75,000_to_99,999","100,000_to_124,999","125,000_to_149,999","150,000_to_199,999","200,000_or_more"
48,2800423,144695,95789,99129,104429,107027,112120,110303,115143,105170,210273,286122,381845,281885,196331,215941,234221


In [56]:
print(f"Average people per household: {income_to_pov.loc[48, 'total']/house_income.loc[48, 'total']}")

Average people per household: 2.5573665121304887


In [66]:
median_house_income = wa.loc[:, 'AJZAE001'].astype(int)
median_house_income = median_house_income[48]
median_house_income

70116

In [68]:
print(f"""HUD income levels:
low (80% median): {median_house_income*.8}
very low (50% median): {median_house_income*.5}
extremely low(30% median): {median_house_income*.3}""")

HUD income levels:
low (80% median): 56092.8
very low (50% median): 35058.0
extremely low(30% median): 21034.8


In [70]:
low_income = (sum(house_income.loc[48, 'less_than_10,000': '45,000_to_49,999']), 
              sum(house_income.loc[48, 'less_than_10,000': '50,000_to_59,999']))

In [69]:
print(f"""Households @ HUD income levels:
low: {low_income[0]} - {low_income[1]}
very low: {sum(house_income.loc[48, 'less_than_10,000': '30,000_to_34,999'])}
extremely low: {sum(house_income.loc[48, 'less_than_10,000': '15,000_to_19,999'])}""")

Households @ HUD income levels:
low: 993805 - 1204078
very low: 663189
extremely low: 339613


In [72]:
print(f"proportion of households that are low income: {low_income[0]/house_income.loc[48, 'total']} - {low_income[1]/house_income.loc[48, 'total']}")

proportion of households that are low income: 0.35487674540596187 - 0.4299629020330143


In [73]:
print(f"people in low income households based on avg household size: {low_income[0]*2.6} - {low_income[1]*2.6}")

people in low income households based on avg household size: 2583893.0 - 3130602.8000000003


## County Level Data

In [81]:
counties = pd.read_csv('../data/2018_county.csv', encoding='latin-1')

In [82]:
wa_counties = counties[counties['STATE'] == 'Washington']

In [83]:
# check for correct county count
wa_counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 2954 to 2992
Columns: 108 entries, GISJOIN to AJ1CM007
dtypes: object(108)
memory usage: 33.2+ KB


In [78]:
wa_counties.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,AJY9M017,AJZAM001,AJ0EM001,AJ1CM001,AJ1CM002,AJ1CM003,AJ1CM004,AJ1CM005,AJ1CM006,AJ1CM007
2954,G5300010,2014-2018,NaN,NaN,Washington,53,Adams County,001,NaN,NaN,...,60,5961,1088,90,298,298,317,181,19,279
2955,G5300030,2014-2018,NaN,NaN,Washington,53,Asotin County,003,NaN,NaN,...,85,2558,1546,97,366,366,410,211,22,370
2956,G5300050,2014-2018,NaN,NaN,Washington,53,Benton County,005,NaN,NaN,...,358,2031,666,290,1074,1060,1091,521,111,1068
2957,G5300070,2014-2018,NaN,NaN,Washington,53,Chelan County,007,NaN,NaN,...,217,2615,1085,202,807,810,814,357,37,826
2958,G5300090,2014-2018,NaN,NaN,Washington,53,Clallam County,009,NaN,NaN,...,192,1998,952,191,758,746,799,274,82,791


In [106]:
c_populations = wa_counties.loc[:,['COUNTY','AJWME001']]
c_populations.columns = ['county', 'population']
c_populations['population'] = c_populations['population'].astype(int)
c_populations

,county,population
2954,Adams County,19452
2955,Asotin County,22337
2956,Benton County,194168
2957,Chelan County,75757
2958,Clallam County,74487
2959,Clark County,465384
2960,Columbia County,4001
2961,Cowlitz County,105112
2962,Douglas County,41371
2963,Ferry County,7576


### Household Income

In [107]:
c_house_income = wa_counties.loc[:,'AJY9E001':'AJY9E017'].astype(int)

In [108]:
c_house_income.columns = ['Total',
        'Less than $10,000',
        '$10,000 to $14,999',
        '$15,000 to $19,999',
        '$20,000 to $24,999',
        '$25,000 to $29,999',
        '$30,000 to $34,999',
        '$35,000 to $39,999',
        '$40,000 to $44,999',
        '$45,000 to $49,999',
        '$50,000 to $59,999',
        '$60,000 to $74,999',
        '$75,000 to $99,999',
        '$100,000 to $124,999',
        '$125,000 to $149,999',
        '$150,000 to $199,999',
        '$200,000 or more']
c_house_income.columns = c_house_income.columns.str.replace(' ', '_').str.replace('$', '').str.lower()

In [109]:
c_populations['avg_household_count'] = c_populations['population']/c_house_income['total']

In [146]:
c_house_income

,total,"less_than_10,000","10,000_to_14,999","15,000_to_19,999","20,000_to_24,999","25,000_to_29,999","30,000_to_34,999","35,000_to_39,999","40,000_to_44,999","45,000_to_49,999","50,000_to_59,999","60,000_to_74,999","75,000_to_99,999","100,000_to_124,999","125,000_to_149,999","150,000_to_199,999","200,000_or_more"
2954,5881,386,335,358,431,343,198,328,409,182,430,701,878,383,185,216,118
2955,9171,528,480,637,390,529,612,485,414,451,876,822,1276,722,300,423,226
2956,70983,3291,2945,2723,3138,3069,2884,2725,2733,2783,5926,8062,9785,7041,4638,4919,4321
2957,28038,1820,947,1138,1575,1418,1528,1123,1870,1209,2174,3198,3356,2433,1523,1516,1210
2958,32732,2034,1952,1935,1691,1943,1906,1639,1782,1510,2637,3393,4161,2289,1472,1428,960
2959,171522,6662,4688,5274,6640,6228,6932,7225,7406,6739,13693,18513,25055,19111,13526,12522,11308
2960,1758,120,123,115,87,102,48,128,63,73,98,206,233,129,62,76,95
2961,41397,2954,1892,2209,2463,2443,2100,2118,1951,1759,3156,4632,5324,3387,1762,2050,1197
2962,15064,555,502,615,528,1060,630,695,814,694,1392,1819,2352,1110,1068,746,484
2963,3097,361,171,208,276,129,174,141,268,93,203,314,306,136,141,125,51


In [110]:
c_populations[['county','avg_household_count']]

,county,avg_household_count
2954,Adams County,3.307601
2955,Asotin County,2.435612
2956,Benton County,2.735416
2957,Chelan County,2.701940
2958,Clallam County,2.275663
2959,Clark County,2.713261
2960,Columbia County,2.275882
2961,Cowlitz County,2.539121
2962,Douglas County,2.746349
2963,Ferry County,2.446238


In [115]:
c_median_house_income.dtypes

county              object
med_house_income    object
dtype: object

In [117]:
c_median_house_income = wa_counties.loc[:, ['COUNTY','AJZAE001']]
c_median_house_income.columns = ['county', 'med_house_income']
c_median_house_income['med_house_income'] = c_median_house_income['med_house_income'].astype(int)
print(c_median_house_income.dtypes)
c_median_house_income['low_threshold'] = .8 * c_median_house_income['med_house_income']
c_median_house_income['v_low_threshold'] = .5 * c_median_house_income['med_house_income']
c_median_house_income['x_low_threshold'] = .3 * c_median_house_income['med_house_income']
c_median_house_income

county              object
med_house_income     int32
dtype: object


,county,med_house_income,low_threshold,v_low_threshold,x_low_threshold
2954,Adams County,49142,39313.6,24571.0,14742.6
2955,Asotin County,50423,40338.4,25211.5,15126.9
2956,Benton County,65650,52520.0,32825.0,19695.0
2957,Chelan County,56135,44908.0,28067.5,16840.5
2958,Clallam County,49913,39930.4,24956.5,14973.9
2959,Clark County,71636,57308.8,35818.0,21490.8
2960,Columbia County,51111,40888.8,25555.5,15333.3
2961,Cowlitz County,51752,41401.6,25876.0,15525.6
2962,Douglas County,60452,48361.6,30226.0,18135.6
2963,Ferry County,41924,33539.2,20962.0,12577.2


In [118]:
c_house_income_thresholds = c_house_income.copy()

In [150]:
# renaming columns as the threshold for inclusion (midpoint)
cols = []
for i, col in enumerate(c_house_income_thresholds.columns):
    splits = col.replace(',', '').split('_')
    income_range = [np.nan, np.nan]
    if splits[0].isnumeric():
        income_range[0] = int(splits[0])
    if len(splits)>1 and splits[2].isnumeric():
        income_range[1] = int(splits[2])
    cols.append(income_range)

In [151]:
cols

[[nan, nan],
 [nan, 10000],
 [10000, 14999],
 [15000, 19999],
 [20000, 24999],
 [25000, 29999],
 [30000, 34999],
 [35000, 39999],
 [40000, 44999],
 [45000, 49999],
 [50000, 59999],
 [60000, 74999],
 [75000, 99999],
 [100000, 124999],
 [125000, 149999],
 [150000, 199999],
 [200000, nan]]

In [256]:
def get_col(n, cols):
    """returns the index and range of the column where n falls within the column range
    params:
      n: value
      cols: list of lists [[col0_min, col0_max], [col1_min, col1_max], [col2_min, col2_max], ...]
    returns:
      (index, range_min, range_max)"""
    for i, col in enumerate(cols):
        if (i == 0) or (n > col[1] and not np.isnan(col[1])):
            continue
        return i

In [236]:
def cal_partial_col(threshold, col_min, col_max, n_hh):
    """returns the number of households proportionate to the location of the threshold within the range of incomes
    """
    return (threshold - col_min) / (col_max - col_min) * n_hh

In [314]:
def est_households(est_hh, threshold, cols, c_house_income):
    est_hh['column_i'] = est_hh[threshold +'_threshold'].apply(lambda x: get_col(x, cols))
    est_hh['col_min'] = [cols[i][0] for i in est_hh['column_i']]
    est_hh['col_max'] = [cols[i][1] for i in est_hh['column_i']]
    est_hh['sum_below'] = [sum(c_house_income.iloc[index, 1:column]) 
                               for index, column in enumerate(est_low_hh['column_i'])]
    est_hh['partial_ratio'] = ((est_hh[threshold +'_threshold'] - est_low_hh['col_min']) /
                             (est_hh['col_max'] - est_low_hh['col_min']))
    est_hh['n_hh_in_band'] = [c_house_income.iloc[index, column] for index, column in enumerate(est_hh['column_i'])]
    est_hh['total_hh_' + threshold] = est_hh['sum_below'] + est_hh['n_hh_in_band']
    return est_hh
# .drop(columns=['column_i', 'col_min', 'col_max', 'sum_below', 'partial_ratio', 'n_hh_in_band'])

In [315]:
est_hh = c_median_house_income.copy()
est_hh = est_households(est_hh, 'low', cols, c_house_income)
# est_hh = est_households(est_hh, 'v_low', cols, c_house_income)
# est_hh = est_households(est_hh, 'x_low', cols, c_house_income)

In [316]:
est_hh['population_low'] = round(est_hh['total_hh_low'] * c_populations['avg_household_count'], 0)
# est_hh['population_v_low'] = round(est_hh['total_hh_v_low'] * c_populations['avg_household_count'], 0)
# est_hh['population_x_low'] = round(est_hh['total_hh_x_low'] * c_populations['avg_household_count'], 0)

In [317]:
est_hh

,county,med_house_income,low_threshold,v_low_threshold,x_low_threshold,column_i,col_min,col_max,sum_below,partial_ratio,n_hh_in_band,total_hh_low,population_low
2954,Adams County,49142,39313.6,24571.0,14742.6,7,35000,39999,2051,0.862893,328,2379,7869.0
2955,Asotin County,50423,40338.4,25211.5,15126.9,8,40000,44999,3661,0.067694,414,4075,9925.0
2956,Benton County,65650,52520.0,32825.0,19695.0,10,50000,59999,26291,0.252025,5926,32217,88127.0
2957,Chelan County,56135,44908.0,28067.5,16840.5,8,40000,44999,9549,0.981796,1870,11419,30853.0
2958,Clallam County,49913,39930.4,24956.5,14973.9,7,35000,39999,11461,0.986277,1639,13100,29811.0
2959,Clark County,71636,57308.8,35818.0,21490.8,10,50000,59999,57794,0.730953,13693,71487,193963.0
2960,Columbia County,51111,40888.8,25555.5,15333.3,8,40000,44999,723,0.177796,63,786,1789.0
2961,Cowlitz County,51752,41401.6,25876.0,15525.6,8,40000,44999,16179,0.280376,1951,18130,46034.0
2962,Douglas County,60452,48361.6,30226.0,18135.6,9,45000,49999,5399,0.672454,694,6093,16734.0
2963,Ferry County,41924,33539.2,20962.0,12577.2,6,30000,34999,1145,0.707982,174,1319,3227.0


In [318]:
print(f"""Total Washingtonians:
low income (<80% of median): {sum(est_hh['population_low'])}
very low income (<50% of median): {sum(est_hh['population_v_low'])}
extra low income (<30% of median): {sum(est_hh['population_x_low'])}""")

KeyError: 'population_v_low'

In [312]:
for index, column in enumerate(est_low_hh['column_i']):
    print(index, column)
    print(c_house_income.iloc[index, 1:column])

0 7
less_than_10,000    386
10,000_to_14,999    335
15,000_to_19,999    358
20,000_to_24,999    431
25,000_to_29,999    343
30,000_to_34,999    198
Name: 2954, dtype: int32
1 8
less_than_10,000    528
10,000_to_14,999    480
15,000_to_19,999    637
20,000_to_24,999    390
25,000_to_29,999    529
30,000_to_34,999    612
35,000_to_39,999    485
Name: 2955, dtype: int32
2 10
less_than_10,000    3291
10,000_to_14,999    2945
15,000_to_19,999    2723
20,000_to_24,999    3138
25,000_to_29,999    3069
30,000_to_34,999    2884
35,000_to_39,999    2725
40,000_to_44,999    2733
45,000_to_49,999    2783
Name: 2956, dtype: int32
3 8
less_than_10,000    1820
10,000_to_14,999     947
15,000_to_19,999    1138
20,000_to_24,999    1575
25,000_to_29,999    1418
30,000_to_34,999    1528
35,000_to_39,999    1123
Name: 2957, dtype: int32
4 7
less_than_10,000    2034
10,000_to_14,999    1952
15,000_to_19,999    1935
20,000_to_24,999    1691
25,000_to_29,999    1943
30,000_to_34,999    1906
Name: 2958, dtype